### Test BDD

In [1]:
import os
import psycopg2
import pandas as pd
import numpy
import sqlalchemy
from sqlalchemy import create_engine 

#### Création BDD

In [2]:
mot_passe = os.environ.get('pg_psw')


In [3]:
conn = psycopg2.connect(
   database="postgres", user='postgres', password=mot_passe, host='localhost', port= '5432'
)
conn.autocommit = True

cursor = conn.cursor()

sql = '''CREATE database ProjetSPQR''';

cursor.execute(sql)
print("Base de données créée avec succès !")

conn.close()


Base de données créée avec succès !


In [4]:
conn = psycopg2.connect(
   database="projetspqr", user='postgres', password=mot_passe, host='localhost', port= '5432'
)
conn.autocommit = True

cursor = conn.cursor()

#### Création des tables

In [5]:
def creer_table(conn, sql_creation_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_creation_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la création de la table")
        print(e)
        return
    cursor.close()
    print("La table a été crée avec succès")

In [6]:
sql_creer_table_personne = """
    CREATE TABLE IF NOT EXISTS personne (
    id bigint,
    nom_fr text,
    praenomen text,
    nomen text,
    cognomen text,
    gens text,
    sexe text,
    date_naissance text,
    lieu_naissance text,
    date_mort text,
    lieu_mort text,
    id_pere bigint,
    id_mere bigint,
    PRIMARY KEY(id)
    );
"""

In [7]:
sql_creer_table_activite = """
    CREATE TABLE IF NOT EXISTS activite (
    personne_id bigint,
    activite text,
    CONSTRAINT fk_personne
        FOREIGN KEY(personne_id)
            REFERENCES personne(id)
    );
"""

In [8]:
sql_creer_table_poste = """
    CREATE TABLE IF NOT EXISTS poste (
    personne_id bigint,
    poste text,
    CONSTRAINT fk_personne
        FOREIGN KEY(personne_id)
            REFERENCES personne(id)
    );
"""

In [9]:
sql_creer_table_mariage = """
    CREATE TABLE IF NOT EXISTS mariage (
    id bigint,
    id_mari bigint,
    id_epouse bigint,
    PRIMARY KEY(id),
    CONSTRAINT fk_personne
        FOREIGN KEY(id_mari)
            REFERENCES personne(id),
        FOREIGN KEY(id_epouse)
            REFERENCES personne(id)
    );
"""

In [10]:
sql_creer_table_oeuvre = """
    CREATE TABLE IF NOT EXISTS oeuvre (
    id bigint,
    titre_fr text,
    titre_lat text,
    auteur_id bigint,
    genre text,
    PRIMARY KEY(id),
    CONSTRAINT fk_personne
        FOREIGN KEY(auteur_id)
            REFERENCES personne(id)
);
"""

In [11]:
creer_table(conn, sql_creer_table_personne)
creer_table(conn, sql_creer_table_activite)
creer_table(conn, sql_creer_table_poste)
creer_table(conn, sql_creer_table_mariage)
creer_table(conn, sql_creer_table_oeuvre)

La table a été crée avec succès
La table a été crée avec succès
La table a été crée avec succès
La table a été crée avec succès
La table a été crée avec succès


#### Dataframes

In [12]:
df_personne = pd.read_table("query_personne.csv", delimiter =",") 
print(df_personne.head()) 

                                     item                   itemLabel  \
0  http://www.wikidata.org/entity/Q727886  Marcus Valerius Maximianus   
1  http://www.wikidata.org/entity/Q728245   Saint Marcian of Syracuse   
2  http://www.wikidata.org/entity/Q728335      Lucius Caninius Gallus   
3  http://www.wikidata.org/entity/Q728349                   Severinus   
4  http://www.wikidata.org/entity/Q728527         Gaius Vibius Marsus   

  praenomenLabel nomenLabel cognomenLabel gensLabel sexeLabel  \
0         Marcus   Valerius           NaN   Valerii  masculin   
1            NaN        NaN           NaN       NaN  masculin   
2         Lucius   Caninius           NaN   Caninii  masculin   
3            NaN    Flavius           NaN    Flavii  masculin   
4          Caius     Vibius        Marsus     Vibii  masculin   

          date_naissance                          lieu_naissance  \
0   0200-01-01T00:00:00Z   http://www.wikidata.org/entity/Q15906   
1   0050-01-01T00:00:00Z  http://w

In [13]:
df_personne.rename(columns={"item": "id",
                            "itemLabel": "nom_fr",
                            "praenomenLabel": "praenomen",
                            "nomenLabel": "nomen",
                            "cognomenLabel": "cognomen",
                            "gensLabel": "gens",
                            "sexeLabel": "sexe",
                            "pere": "id_pere",
                            "mere": "id_mere"},
                  inplace=True)

In [14]:
print(df_personne.head())

                                       id                      nom_fr  \
0  http://www.wikidata.org/entity/Q727886  Marcus Valerius Maximianus   
1  http://www.wikidata.org/entity/Q728245   Saint Marcian of Syracuse   
2  http://www.wikidata.org/entity/Q728335      Lucius Caninius Gallus   
3  http://www.wikidata.org/entity/Q728349                   Severinus   
4  http://www.wikidata.org/entity/Q728527         Gaius Vibius Marsus   

  praenomen     nomen cognomen     gens      sexe         date_naissance  \
0    Marcus  Valerius      NaN  Valerii  masculin   0200-01-01T00:00:00Z   
1       NaN       NaN      NaN      NaN  masculin   0050-01-01T00:00:00Z   
2    Lucius  Caninius      NaN  Caninii  masculin  -0100-01-01T00:00:00Z   
3       NaN   Flavius      NaN   Flavii  masculin   0450-01-01T00:00:00Z   
4     Caius    Vibius   Marsus    Vibii  masculin  -0100-01-01T00:00:00Z   

                           lieu_naissance              date_mort  \
0   http://www.wikidata.org/entity/Q

In [15]:
df_personne['id'] = df_personne.id.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)
df_personne['lieu_naissance'] = df_personne.lieu_naissance.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)
df_personne['lieu_mort'] = df_personne.lieu_mort.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)
df_personne['id_pere'] = df_personne.id_pere.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)
df_personne['id_mere'] = df_personne.id_mere.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)

In [16]:
df_personne

,id,nom_fr,praenomen,nomen,cognomen,gens,sexe,date_naissance,lieu_naissance,date_mort,lieu_mort,id_pere,id_mere
0,727886,Marcus Valerius Maximianus,Marcus,Valerius,NaN,Valerii,masculin,0200-01-01T00:00:00Z,15906,t2303561388,NaN,NaN,NaN
1,728245,Saint Marcian of Syracuse,NaN,NaN,NaN,NaN,masculin,0050-01-01T00:00:00Z,200441,0068-01-01T00:00:00Z,13670,NaN,NaN
2,728335,Lucius Caninius Gallus,Lucius,Caninius,NaN,Caninii,masculin,-0100-01-01T00:00:00Z,NaN,-0060-01-01T00:00:00Z,NaN,1385959,NaN
3,728349,Severinus,NaN,Flavius,NaN,Flavii,masculin,0450-01-01T00:00:00Z,NaN,NaN,NaN,NaN,NaN
4,728527,Gaius Vibius Marsus,Caius,Vibius,Marsus,Vibii,masculin,-0100-01-01T00:00:00Z,NaN,-0060-01-01T00:00:00Z,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8888,106225794,Quinctius,t2273118074,Quinctius,NaN,Quinctii,masculin,NaN,NaN,NaN,NaN,NaN,NaN
8889,106225817,Tiberius Q[uinctius?],Tiberius,Quinctius,NaN,Quinctii,masculin,NaN,NaN,NaN,NaN,NaN,NaN
8890,106225852,Q[uinctius?],t2273141310,Quinctius,NaN,Quinctii,masculin,NaN,NaN,NaN,NaN,NaN,NaN
8891,106261999,Caius Duillius Longus,Caius,Duilius,Longus,Duilii,masculin,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_personne.sort_values("id")

,id,nom_fr,praenomen,nomen,cognomen,gens,sexe,date_naissance,lieu_naissance,date_mort,lieu_mort,id_pere,id_mere
8663,100145963,Julius Severus,t2294460433,Julius,Severus,Julii,masculin,NaN,NaN,NaN,NaN,NaN,NaN
1403,1001933,Hélène,NaN,NaN,NaN,NaN,féminin,0301-01-01T00:00:00Z,NaN,0360-11-01T00:00:00Z,26849,8413,231063
1404,1001940,Flavius Florentius,NaN,Flavius,NaN,Flavii,masculin,0350-01-01T00:00:00Z,NaN,NaN,NaN,NaN,NaN
1405,1001947,Mallius Theodorus,t2204777134,Mallius,NaN,Mallii,masculin,0350-01-01T00:00:00Z,729978,NaN,NaN,NaN,NaN
1406,1001949,Charisius,NaN,NaN,NaN,NaN,masculin,0400-01-01T00:00:00Z,15,0400-01-01T00:00:00Z,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8657,99726753,Cassia Lepida,NaN,Cassia,Lepida,Cassii,féminin,NaN,NaN,NaN,NaN,164923,NaN
8659,99774707,Q99774707,Caeso,Fabius,Vibulanus,Fabii,masculin,-0600-01-01T00:00:00Z,1747689,t2095191322,t2095191327,t2095191320,t2095191319
8660,99844311,Caecilia Metella,NaN,Caecilia,Metella,Caecilii,féminin,NaN,1747689,NaN,t2234125828,355768,t2234125829
8661,99849517,Marco Fabio Vibulano,Marcus,Fabius,Vibulanus,Fabii,masculin,t2223603935,1747689,-0500-01-01T00:00:00Z,t2223603931,1232953,t2223603930


In [18]:
df_personne.drop_duplicates(subset="id", keep="first", inplace=True)

In [19]:
df_personne

,id,nom_fr,praenomen,nomen,cognomen,gens,sexe,date_naissance,lieu_naissance,date_mort,lieu_mort,id_pere,id_mere
0,727886,Marcus Valerius Maximianus,Marcus,Valerius,NaN,Valerii,masculin,0200-01-01T00:00:00Z,15906,t2303561388,NaN,NaN,NaN
1,728245,Saint Marcian of Syracuse,NaN,NaN,NaN,NaN,masculin,0050-01-01T00:00:00Z,200441,0068-01-01T00:00:00Z,13670,NaN,NaN
2,728335,Lucius Caninius Gallus,Lucius,Caninius,NaN,Caninii,masculin,-0100-01-01T00:00:00Z,NaN,-0060-01-01T00:00:00Z,NaN,1385959,NaN
3,728349,Severinus,NaN,Flavius,NaN,Flavii,masculin,0450-01-01T00:00:00Z,NaN,NaN,NaN,NaN,NaN
4,728527,Gaius Vibius Marsus,Caius,Vibius,Marsus,Vibii,masculin,-0100-01-01T00:00:00Z,NaN,-0060-01-01T00:00:00Z,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8888,106225794,Quinctius,t2273118074,Quinctius,NaN,Quinctii,masculin,NaN,NaN,NaN,NaN,NaN,NaN
8889,106225817,Tiberius Q[uinctius?],Tiberius,Quinctius,NaN,Quinctii,masculin,NaN,NaN,NaN,NaN,NaN,NaN
8890,106225852,Q[uinctius?],t2273141310,Quinctius,NaN,Quinctii,masculin,NaN,NaN,NaN,NaN,NaN,NaN
8891,106261999,Caius Duillius Longus,Caius,Duilius,Longus,Duilii,masculin,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_personne2 = df_personne.replace(to_replace ='t..........', value = '', regex = True)


In [21]:
df_personne2

,id,nom_fr,praenomen,nomen,cognomen,gens,sexe,date_naissance,lieu_naissance,date_mort,lieu_mort,id_pere,id_mere
0,727886,Marcus Valerius Maximianus,Marcus,Valerius,NaN,Valerii,masculin,0200-01-01T00:00:00Z,15906,,NaN,NaN,NaN
1,728245,Sainf Syracuse,NaN,NaN,NaN,NaN,masculin,0050-01-01T00:00:00Z,200441,0068-01-01T00:00:00Z,13670,NaN,NaN
2,728335,Lucius Caninius Gallus,Lucius,Caninius,NaN,Caninii,masculin,-0100-01-01T00:00:00Z,NaN,-0060-01-01T00:00:00Z,NaN,1385959,NaN
3,728349,Severinus,NaN,Flavius,NaN,Flavii,masculin,0450-01-01T00:00:00Z,NaN,NaN,NaN,NaN,NaN
4,728527,Gaius Vibius Marsus,Caius,Vibius,Marsus,Vibii,masculin,-0100-01-01T00:00:00Z,NaN,-0060-01-01T00:00:00Z,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8888,106225794,Quinctius,,Quinctius,NaN,Quinctii,masculin,NaN,NaN,NaN,NaN,NaN,NaN
8889,106225817,Tiberius Q[uinctius?],Tiberius,Quinctius,NaN,Quinctii,masculin,NaN,NaN,NaN,NaN,NaN,NaN
8890,106225852,Q[uinctius?],,Quinctius,NaN,Quinctii,masculin,NaN,NaN,NaN,NaN,NaN,NaN
8891,106261999,Caius Duillius Longus,Caius,Duilius,Longus,Duilii,masculin,NaN,NaN,NaN,NaN,NaN,NaN


##### activite

In [22]:
df_activite = pd.read_table("query_activite.csv", delimiter =",") 
print(df_activite.head()) 

                                    item         occupationLabel
0  http://www.wikidata.org/entity/Q46762               militaire
1  http://www.wikidata.org/entity/Q46762  personnalité politique
2  http://www.wikidata.org/entity/Q46768  personnalité politique
3  http://www.wikidata.org/entity/Q46780  personnalité politique
4  http://www.wikidata.org/entity/Q46814  personnalité politique


In [23]:
df_activite.rename(columns={"item": "personne_id",
                            "occupationLabel": "activite"},
                  inplace=True)

In [24]:
df_activite['personne_id'] = df_activite.personne_id.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)

In [25]:
print(df_activite.head()) 

  personne_id                activite
0       46762               militaire
1       46762  personnalité politique
2       46768  personnalité politique
3       46780  personnalité politique
4       46814  personnalité politique


In [26]:
df_activite2 = df_activite.replace(to_replace ='^t.+', value = '', regex = True)


In [27]:
df_activite2

,personne_id,activite
0,46762,militaire
1,46762,personnalité politique
2,46768,personnalité politique
3,46780,personnalité politique
4,46814,personnalité politique
...,...,...
9822,1558527,soldat
9823,106489809,
9824,106510599,NaN
9825,106510646,NaN


##### poste

In [28]:
df_poste = pd.read_table("query_poste.csv", delimiter =",") 
print(df_poste.head()) 

                                   item     positionLabel
0  http://www.wikidata.org/entity/Q1048            consul
1  http://www.wikidata.org/entity/Q1048           préteur
2  http://www.wikidata.org/entity/Q1048          questeur
3  http://www.wikidata.org/entity/Q1048  pontifex maximus
4  http://www.wikidata.org/entity/Q1048         dictateur


In [29]:
df_poste.rename(columns={"item": "personne_id",
                            "positionLabel": "poste"},
                  inplace=True)

In [30]:
df_poste['personne_id'] = df_poste.personne_id.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)

In [31]:
print(df_poste.head()) 

  personne_id             poste
0        1048            consul
1        1048           préteur
2        1048          questeur
3        1048  pontifex maximus
4        1048         dictateur


In [32]:
df_poste2 = df_poste.replace(to_replace ='t.+', value = '', regex = True)


In [33]:
df_poste2

,personne_id,poste
0,1048,consul
1,1048,pré
2,1048,ques
3,1048,pon
4,1048,dic
...,...,...
10672,98381019,préfe
10673,98381019,préfe
10674,98398735,NaN
10675,98446725,NaN


##### oeuvre

In [34]:
df_oeuvre = pd.read_table("query_oeuvre.csv", delimiter =",") 
print(df_oeuvre.head()) 

                                      oeuvre  \
0   http://www.wikidata.org/entity/Q87213120   
1     http://www.wikidata.org/entity/Q660519   
2     http://www.wikidata.org/entity/Q932676   
3   http://www.wikidata.org/entity/Q19074050   
4  http://www.wikidata.org/entity/Q104789146   

                                         oeuvreLabel                  titre  \
0  Abrégé des Histoires philippiques de Trogue Po...                    NaN   
1                                     Noctes Atticae         Noctes Atticae   
2                                        Diatessaron                    NaN   
3                              Address to the Greeks  Ἐπιστολὴ πρὸς Ἕλληνας   
4                                       Controverses                    NaN   

                                   auteur    genreLabel  
0  http://www.wikidata.org/entity/Q210853           NaN  
1  http://www.wikidata.org/entity/Q294800      Q1009974  
2  http://www.wikidata.org/entity/Q272087           NaN  
3  h

In [35]:
df_oeuvre.rename(columns={"oeuvre": "id",
                          "oeuvreLabel": "titre_fr",
                          "titre": "titre_lat",
                          "auteur": "auteur_id",
                          "genreLabel": "genre"},
                  inplace=True)

In [36]:
for index, row in df_oeuvre.iterrows():
    e = df_oeuvre.at[index, 'id']
    e.replace('http://www.wikidata.org/entity/Q', '')

In [37]:
print(df_oeuvre.head()) 

                                          id  \
0   http://www.wikidata.org/entity/Q87213120   
1     http://www.wikidata.org/entity/Q660519   
2     http://www.wikidata.org/entity/Q932676   
3   http://www.wikidata.org/entity/Q19074050   
4  http://www.wikidata.org/entity/Q104789146   

                                            titre_fr              titre_lat  \
0  Abrégé des Histoires philippiques de Trogue Po...                    NaN   
1                                     Noctes Atticae         Noctes Atticae   
2                                        Diatessaron                    NaN   
3                              Address to the Greeks  Ἐπιστολὴ πρὸς Ἕλληνας   
4                                       Controverses                    NaN   

                                auteur_id         genre  
0  http://www.wikidata.org/entity/Q210853           NaN  
1  http://www.wikidata.org/entity/Q294800      Q1009974  
2  http://www.wikidata.org/entity/Q272087           NaN  
3  h

In [38]:
df_oeuvre['id'] = df_oeuvre.id.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)
df_oeuvre['auteur_id'] = df_oeuvre.auteur_id.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)

In [39]:
df_oeuvre

,id,titre_fr,titre_lat,auteur_id,genre
0,87213120,Abrégé des Histoires philippiques de Trogue Po...,NaN,210853,NaN
1,660519,Noctes Atticae,Noctes Atticae,294800,Q1009974
2,932676,Diatessaron,NaN,272087,NaN
3,19074050,Address to the Greeks,Ἐπιστολὴ πρὸς Ἕλληνας,272087,NaN
4,104789146,Controverses,NaN,273671,controversia
...,...,...,...,...,...
132,15804866,Sur les maladies aiguës et les maladies chroni...,De morbis acutis et chronicis,2514828,NaN
133,42187365,Lives of the Sophists,NaN,2274948,NaN
134,2403439,fragments de Quadrat,NaN,12879156,NaN
135,44227030,Passion de Perpétue et de Félicité,Passio sanctarum Perpetuae et Felicitatis,18061415,hagiographie


In [40]:
df_oeuvre.drop_duplicates(subset="id", keep="first", inplace=True)

In [41]:
df_oeuvre

,id,titre_fr,titre_lat,auteur_id,genre
0,87213120,Abrégé des Histoires philippiques de Trogue Po...,NaN,210853,NaN
1,660519,Noctes Atticae,Noctes Atticae,294800,Q1009974
2,932676,Diatessaron,NaN,272087,NaN
3,19074050,Address to the Greeks,Ἐπιστολὴ πρὸς Ἕλληνας,272087,NaN
4,104789146,Controverses,NaN,273671,controversia
...,...,...,...,...,...
132,15804866,Sur les maladies aiguës et les maladies chroni...,De morbis acutis et chronicis,2514828,NaN
133,42187365,Lives of the Sophists,NaN,2274948,NaN
134,2403439,fragments de Quadrat,NaN,12879156,NaN
135,44227030,Passion de Perpétue et de Félicité,Passio sanctarum Perpetuae et Felicitatis,18061415,hagiographie


In [42]:
df_oeuvre2 = df_oeuvre.replace(to_replace ='^t.+', value = '', regex = True)


In [43]:
df_oeuvre2

,id,titre_fr,titre_lat,auteur_id,genre
0,87213120,Abrégé des Histoires philippiques de Trogue Po...,NaN,210853,NaN
1,660519,Noctes Atticae,Noctes Atticae,294800,Q1009974
2,932676,Diatessaron,NaN,272087,NaN
3,19074050,Address to the Greeks,Ἐπιστολὴ πρὸς Ἕλληνας,272087,NaN
4,104789146,Controverses,NaN,273671,controversia
...,...,...,...,...,...
132,15804866,Sur les maladies aiguës et les maladies chroni...,De morbis acutis et chronicis,2514828,NaN
133,42187365,Lives of the Sophists,NaN,2274948,NaN
134,2403439,fragments de Quadrat,NaN,12879156,NaN
135,44227030,Passion de Perpétue et de Félicité,Passio sanctarum Perpetuae et Felicitatis,18061415,hagiographie


##### mariage

In [44]:
df_mariage = pd.read_table("query_mariage.csv", delimiter =",") 
print(df_mariage.head()) 

                                     item  \
0  http://www.wikidata.org/entity/Q170026   
1  http://www.wikidata.org/entity/Q170512   
2  http://www.wikidata.org/entity/Q171023   
3  http://www.wikidata.org/entity/Q171083   
4  http://www.wikidata.org/entity/Q172168   

                                    epouse  
0   http://www.wikidata.org/entity/Q232329  
1                               t291183301  
2   http://www.wikidata.org/entity/Q383304  
3  http://www.wikidata.org/entity/Q4066696  
4   http://www.wikidata.org/entity/Q234562  


In [45]:
df_mariage.insert(0, 'id', range(0, len(df_mariage)))


In [46]:
df_mariage.rename(columns={"item": "id_mari",
                          "epouse": "id_epouse"},
                  inplace=True)

In [47]:
print(df_mariage.head())

   id                                 id_mari  \
0   0  http://www.wikidata.org/entity/Q170026   
1   1  http://www.wikidata.org/entity/Q170512   
2   2  http://www.wikidata.org/entity/Q171023   
3   3  http://www.wikidata.org/entity/Q171083   
4   4  http://www.wikidata.org/entity/Q172168   

                                 id_epouse  
0   http://www.wikidata.org/entity/Q232329  
1                               t291183301  
2   http://www.wikidata.org/entity/Q383304  
3  http://www.wikidata.org/entity/Q4066696  
4   http://www.wikidata.org/entity/Q234562  


In [48]:
df_mariage['id_mari'] = df_mariage.id_mari.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)
df_mariage['id_epouse'] = df_mariage.id_epouse.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)

In [49]:
print(df_mariage.head()) 

   id id_mari   id_epouse
0   0  170026      232329
1   1  170512  t291183301
2   2  171023      383304
3   3  171083     4066696
4   4  172168      234562


In [50]:
df_mariage2 = df_mariage.replace(to_replace ='^t.+', value = '', regex = True)


In [51]:
df_mariage2

,id,id_mari,id_epouse
0,0,170026,232329
1,1,170512,
2,2,171023,383304
3,3,171083,4066696
4,4,172168,234562
...,...,...,...
1202,1202,64831801,
1203,1203,64843265,
1204,1204,64853920,
1205,1205,64868893,64854646


#### Insertion

In [52]:
sql_insert_personne = """INSERT INTO personne (id, nom_fr, praenomen, nomen, cognomen, gens, sexe, date_naissance, lieu_naissance, date_mort, lieu_mort, id_pere, id_mere) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

In [53]:
sql_insert_oeuvre = """INSERT INTO oeuvre (id, titre_fr, titre_lat, auteur_id, genre) VALUES (%s, %s, %s, %s, %s)"""

In [54]:
sql_insert_poste = """INSERT INTO poste (personne_id, poste) VALUES (%s, %s)"""

In [55]:
sql_insert_activite = """INSERT INTO activite (personne_id, activite) VALUES (%s, %s)"""

In [56]:
sql_insert_mariage = """INSERT INTO mariage (id, id_mari, id_epouse) VALUES (%s, %s, %s)"""

In [57]:
conn = psycopg2.connect(
   database="projetspqr", user='postgres', password=mot_passe, host='localhost', port= '5432'
)
conn.autocommit = True

cursor = conn.cursor()

for index, row in df_personne.iterrows():
    cursor.execute(sql_insert_personne, tuple(row))

NumericValueOutOfRange: ERREUR:  bigint en dehors des limites


In [ ]:
for index, row in df_oeuvre.iterrows():
    cursor.execute(sql_insert_oeuvre, tuple(row))

In [ ]:
for index, row in df_activite.iterrows():
    cursor.execute(sql_insert_activite, tuple(row))


In [ ]:
for index, row in df_poste.iterrows():
    cursor.execute(sql_insert_poste, tuple(row))


In [ ]:
for index, row in df_mariage.iterrows():
    cursor.execute(sql_insert_mariage, tuple(row))

In [ ]:
df_mariage.drop(df_mariage.index[df_mariage['id_epouse'] == 't291183301'], inplace = True)

In [ ]:
df_mariage

In [ ]:
print(df_mariage[2])